In [21]:
##Imports
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

In [22]:
##Store CSV into DataFrame

csv_file = "../Data/Netflix_titles.csv"
netflix_df = pd.read_csv(csv_file)
netflix_df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7647,s7362,TV Show,Unsolved Mysteries,NaN,NaN,NaN,19-Oct-20,2020,TV-MA,2 Seasons,"Crime TV Shows, Docuseries","Real cases of perplexing disappearances, shock..."
7653,s7383,TV Show,Vagabond,NaN,"Lee Seung-gi, Bae Suzy, Shin Sung-rok, Moon Ju...",South Korea,21-Sep-19,2019,TV-MA,1 Season,"International TV Shows, Korean TV Shows, TV Ac...","When his nephew dies in a plane crash, stunt m..."
865,s1178,Movie,Calico Critters: The Treasure of Calico Village,Momoko Kamiya,NaN,NaN,1-Nov-18,2017,TV-Y,22 min,Children & Family Movies,Freya and her friends inspire the mayor to put...
2213,s3172,Movie,Jim Jefferies : BARE,Shannon Hartman,Jim Jefferies,United States,6-Aug-19,2014,TV-MA,77 min,Stand-Up Comedy,Nothing is sacred in this show from Australian...
2652,s3803,Movie,Luccas Neto in: Summer Camp,Lucas Margutti,"Luccas Neto, Gi Alparone, Jéssica Diehl, Roni ...",Brazil,5-Nov-19,2019,TV-G,87 min,"Children & Family Movies, Comedies, Music & Mu...",Luccas and Gi are heading to a world famous gy...


In [23]:
##Read Columns

for col in netflix_df.columns: 
    print(col)
    

show_id
type
title
director
cast
country
date_added
release_year
rating
duration
listed_in
description


In [24]:
##Drop all non-Movies and only capture U.S, and movies 2000-2017
netflix_df = netflix_df[(netflix_df.type == "Movie") & (netflix_df.release_year > 1999) & (netflix_df.release_year < 2018)] 
netflix_df.sample(5)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1121,s1583,Movie,Dagh Ujala,Raj Amit Kumar,"Victor Banerjee, Adil Hussain, Bhanu Uday, Pre...","United States, India",1-Apr-18,2015,TV-MA,103 min,"Dramas, Independent Movies, International Movies",A lesbian daughter fights off an arranged marr...
4091,s5887,Movie,Sunrise,Partho Sen-Gupta,"Adil Hussain, Tannishtha Chatterjee, Gulnaaz A...","India, France",15-Mar-17,2014,TV-MA,85 min,"Dramas, Independent Movies, International Movies","While investigating a child kidnapping, a Mumb..."
265,s331,Movie,Aftermath,Elliott Lester,"Arnold Schwarzenegger, Scoot McNairy, Maggie G...","United Kingdom, United States",15-Mar-20,2017,R,94 min,Dramas,After an air traffic controller's mistake resu...
1841,s2640,Movie,Harud,Aamir Bashir,"Mohammad Amir Naji, Shahnawaz Bhat, Shamim Bas...",India,1-Oct-16,2010,TV-MA,100 min,"Dramas, International Movies",A young Kashmiri man sets out to escape to Pak...
1501,s2150,Movie,Ferrari Ki Sawaari,Rajesh Mapuskar,"Sharman Joshi, Boman Irani, Ritwik Sahore, Par...",India,1-Aug-19,2012,TV-PG,133 min,"Comedies, International Movies, Sports Movies",A father winds up involved with a stolen Ferra...


In [33]:
##Choose Columns

clean_netflix_df = netflix_df[['title', 'director', 'cast', 'release_year', 'rating', 'duration', 'listed_in', 'description']].copy()
clean_netflix_df = clean_netflix_df.rename(columns={'title': 'movie_title'})

clean_netflix_df = clean_netflix_df.reset_index(drop=True)
clean_netflix_df.head(5)

clean_netflix_df.to_csv("../Output/Netflix_clean.csv")


In [26]:
#Connect to Local Database

rds_connection_string = "postgres:postgres@localhost:5432/ETL_project_movie_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [27]:
#Check for tables
engine.table_names()

['netflix_data', 'IMBD_data']

In [28]:
### Use pandas to load csv converted DataFrame into database
clean_netflix_df.to_sql(name='netflix_data', con=engine, if_exists='append', index=False)

In [29]:
##Confirm data has been added by querying the netflix_data table
pd.read_sql_query('select * from netflix_data', con=engine).head()

,movie_title,director,cast,release_year,rating,duration,listed_in,description
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
4,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",2008,TV-MA,143 min,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...
